In [1]:
import pandas as pd
import numpy as np

In [3]:
# 1. 导入数据
chipo = pd.read_table("../data/chipotle.tsv", sep="\t") # tsv文件就是制表符分割
# 2. 查看前10行数据
chipo.head(10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


In [4]:
# 3. 查看数据有多少列
chipo.columns.size

5

In [5]:
# 4. 打印全部列名
chipo.columns

Index(['order_id', 'quantity', 'item_name', 'choice_description',
       'item_price'],
      dtype='object')

In [6]:
# 5. 查看数据集的索引
chipo.index

RangeIndex(start=0, stop=4622, step=1)

In [70]:
# 6. 被下单数最多的商品item是什么
# items = chipo[['quantity', 'item_name']].groupby(by='item_name') # 先分组
# print(items.count()) # 查看各商品被下单的次数

items_pt = pd.pivot_table(chipo[['quantity', 'item_name']], index='item_name', aggfunc=np.sum) # 创建透视表
print(items_pt) # 查看创建的透视表
print(items_pt['quantity'].max())
# print(items_pt['quantity']==items_pt['quantity'].max()) 返回一个DataFrame 只有满足条件的那一行对应的值为True
print(items_pt[items_pt['quantity']==items_pt['quantity'].max()]) # 找到下单次数最多的这一行
print(items_pt[items_pt['quantity']==items_pt['quantity'].max()].index) # 由于索引名为商品名
items_pt[items_pt['quantity']==items_pt['quantity'].max()].index[0] # 提取出该商品名

                                       quantity
item_name                                      
6 Pack Soft Drink                            55
Barbacoa Bowl                                66
Barbacoa Burrito                             91
Barbacoa Crispy Tacos                        12
Barbacoa Salad Bowl                          10
Barbacoa Soft Tacos                          25
Bottled Water                               211
Bowl                                          4
Burrito                                       6
Canned Soda                                 126
Canned Soft Drink                           351
Carnitas Bowl                                71
Carnitas Burrito                             60
Carnitas Crispy Tacos                         8
Carnitas Salad                                1
Carnitas Salad Bowl                           6
Carnitas Soft Tacos                          40
Chicken Bowl                                761
Chicken Burrito                         

'Chicken Bowl'

In [71]:
# 7. 在item_name这一列，一共有多少种商品被下单
print(chipo[chipo['quantity']==0]) # 先确定是否存在商品没下单的情况，结果显示并没有这种情况
chipo['item_name'].unique().size # unique()去重

Empty DataFrame
Columns: [order_id, quantity, item_name, choice_description, item_price]
Index: []


50

In [74]:
# 8. 一共有多少个商品被下单
print(chipo['quantity'].sum())
items_pt.sum() # 通过上面的透视表进行确认

4972


quantity    4972
dtype: int64

In [88]:
# 9. 将item_price转换为浮点数
print(chipo['item_price'])
chipo['item_price'].str.extract('(\d+\.\d+)') # 通过正则提取出除 $ 符以外的数字
print(chipo['item_price'].str[1:]) # 等同于上面，这样操作返回的是一个DataFrame
chipo['item_price'] = chipo['item_price'].str[1:].astype(float) # 转换为浮点型，同时更新进原数据集
chipo['item_price']

0        $2.39 
1        $3.39 
2        $3.39 
3        $2.39 
4       $16.98 
         ...   
4617    $11.75 
4618    $11.75 
4619    $11.25 
4620     $8.75 
4621     $8.75 
Name: item_price, Length: 4622, dtype: object
0        2.39 
1        3.39 
2        3.39 
3        2.39 
4       16.98 
         ...  
4617    11.75 
4618    11.75 
4619    11.25 
4620     8.75 
4621     8.75 
Name: item_price, Length: 4622, dtype: object


0        2.39
1        3.39
2        3.39
3        2.39
4       16.98
        ...  
4617    11.75
4618    11.75
4619    11.25
4620     8.75
4621     8.75
Name: item_price, Length: 4622, dtype: float64

In [91]:
# 10. 在数据集对应的时期内，收入(revenue)是多少？
# 由于同款商品买的价钱不一样，所以应该用每行quantity * item_price，再累加起来就是最终的revenue
revenue = (chipo['quantity'] * chipo['item_price']).sum()
revenue

39237.02

In [102]:
# 11. 一共有多少订单？
# 一个order_id就是一个订单，由于上面已经验证不存在quantity为0的情况，所以直接统计有多少个order_id即可
print(chipo['order_id'].unique()[-1]) # 即为订单总数

1834


In [112]:
# 12. 每一单order对应的平均总价是多少？
# 先分组，总聚合
order_group = chipo[['order_id', 'quantity', 'item_price']].groupby(by='order_id') # 先分组
order_agg = order_group.agg(np.sum) # 再聚合，聚合时，统计出每单有多少个商品。多少个商品合计多少钱
order_agg['avg_price'] = order_agg['item_price'] / order_agg['quantity'] # 添加一列，为每单对应的平均总价
order_agg

,quantity,item_price,avg_price
order_id,,,
1,4,11.56,2.890000
2,2,16.98,8.490000
3,2,12.67,6.335000
4,2,21.00,10.500000
5,2,13.70,6.850000
...,...,...,...
1830,2,23.00,11.500000
1831,3,12.90,4.300000
1832,2,13.20,6.600000
